In [ ]:
# !pip install -r requirements.txt

  Using cached aiohappyeyeballs-2.4.3-py3-none-any.whl (14 kB)
  Using cached aiohttp-3.10.10-cp310-cp310-macosx_11_0_arm64.whl (390 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached alembic-1.14.0-py3-none-any.whl (233 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
  Using cached anthropic-0.39.0-py3-none-any.whl (198 kB)
  Using cached anyio-4.6.2.post1-py3-none-any.whl (90 kB)
  Using cached asgiref-3.8.1-py3-none-any.whl (23 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl (5.7 kB)
  Using cached attrs-24.2.0-py3-none-any.whl (63 kB)
  Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
  Using cached bcrypt-4.2.0-cp39-abi3-macosx_10_12_universal2.whl (472 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
  Using cached build-1.2.2.post1-py3-none-any.whl (22 kB)
  Using cached cachetools-5.5.0-py3-none-any.whl (9.5 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl (167 kB)
  Using cached charset_normalizer-3.4

In [2]:
import pandas as pd
med_qa_data = pd.read_csv('data/MedQA_complete_graded_data_encoded.csv')
med_qa_data = med_qa_data.drop(columns = ['index', 'Unnamed: 29', 'Unnamed: 30'])

In [3]:
med_qa_data.head()

,question,answer,new_IR,IR Grade,new_AR,AR Grade,new_BR,BR Grade,new_DR,DR Grade,...,DR Grade.1,IR,Rationale.2,IR Grade.1,AR,Rationale.3,AR Grade.1,BR,Rationale.4,BR Grade.1
0,Two weeks after undergoing an emergency cardia...,Cholesterol embolization,Thrombotic microangiopathy,N,Thrombotic microangiopathy.,N,Thrombotic microangiopathy.,N,Thrombotic microangiopathy,N,...,Y,Atheroembolic renal disease,"This patient has decreased urinary output, mal...",Y,Cholesterol embolism,The patient's recent history of cardiac cathet...,Y,Contrast-induced nephropathy,This patient's symptoms and clinical history s...,N
1,A 46-year-old man is brought to the emergency ...,Benzodiazepine intoxication,Alcohol intoxication,N,Alcohol intoxication,N,Alcohol intoxication,N,Alcohol intoxication,N,...,N,Wernicke's encephalopathy,This patient's symptoms of altered mental stat...,N,Wernicke's encephalopathy,"The patient's symptoms (somnolence, slurred sp...",N,Wernicke's encephalopathy,The prior probability of Wernicke's encephalop...,N
2,A 30-year-old African American woman comes to ...,Histoplasma capsulatum infection,Valley Fever,N,Coccidioidomycosis,N,Histoplasmosis,Y,Fungal infection,N,...,Y,Histoplasmosis,"This patient presents with fever, cough, chest...",Y,Histoplasmosis,The patient's history of recent hiking in Miss...,Y,Histoplasma capsulatum,The prior probability of Histoplasmosis (a fun...,Y
3,A 67-year-old man who was diagnosed with arthr...,Psoriatic arthritis,Psoriatic arthritis,Y,Psoriatic arthritis,Y,Psoriatic arthritis,Y,Psoriatic arthritis,Y,...,Y,Psoriatic Arthritis,The patient's clinical presentation suggests p...,Y,Psoriatic Arthritis,The patient presents with several features sug...,Y,Psoriatic Arthritis,The prior probability of psoriatic arthritis i...,Y
4,A one-day-old male is evaluated in the hospita...,Duodenal atresia,Gastroschisis,N,Intestinal obstruction,N,Gastroschisis,N,Duodenal atresia,Y,...,Y,Meconium ileus,This neonate's clinical presentation of biliou...,N,Intestinal atresia,"The symptoms presented by this neonate, such a...",Y,Duodenal atresia in Down Syndrome,"In Bayesian reasoning, we begin with a prior p...",Y


In [4]:
med_qa_data['answer'].str.replace('/', ' ')

0                      Cholesterol embolization
1                   Benzodiazepine intoxication
2              Histoplasma capsulatum infection
3                           Psoriatic arthritis
4                              Duodenal atresia
                         ...                   
508                    Chronic cerebral hypoxia
509                           Kallmann syndrome
510    Attention-deficit hyperactivity disorder
511                           Bronchial rupture
512                                         NaN
Name: answer, Length: 513, dtype: object

## Data cleaning and such

In [5]:
import re

# replace newline characters and " characters
med_qa_data['answer'] = med_qa_data['answer'].str.replace('\n', '')
med_qa_data['answer'] = med_qa_data['answer'].str.replace('"', '')

def remove_non_alphanumeric(input_string):
    if isinstance(input_string, str):
        return re.sub(r'[^a-zA-Z0-9 ]', ' ', input_string)
    return input_string

med_qa_data['answer'] = med_qa_data['answer'].apply(lambda row: remove_non_alphanumeric(row))

In [6]:
med_qa_data[med_qa_data['answer'].apply(lambda x: isinstance(x, str) and '"' in x)]

,question,answer,new_IR,IR Grade,new_AR,AR Grade,new_BR,BR Grade,new_DR,DR Grade,...,DR Grade.1,IR,Rationale.2,IR Grade.1,AR,Rationale.3,AR Grade.1,BR,Rationale.4,BR Grade.1


In [7]:
med_qa_data.tail()

,question,answer,new_IR,IR Grade,new_AR,AR Grade,new_BR,BR Grade,new_DR,DR Grade,...,DR Grade.1,IR,Rationale.2,IR Grade.1,AR,Rationale.3,AR Grade.1,BR,Rationale.4,BR Grade.1
508,A previously healthy 21-year-old man comes to ...,Chronic cerebral hypoxia,Carbon monoxide poisoning,Y,Mold exposure,N,Carbon monoxide poisoning,Y,Anemia,N,...,Y,"This young man presents with headache, letharg...",Radon exposure,N,"This patient's symptoms, especially headache, ...",Chronic hypoxia-induced polycythemia.,Y,"The starting point, or the prior probability, ...",Chronic carbon monoxide exposure.,Y
509,A 15-year-old boy is brought to the physician ...,Kallmann syndrome,Klinefelter Syndrome,N,Delayed puberty,N,Delayed puberty.,N,Delayed puberty,N,...,N,This patient's delayed puberty (Tanner stage 1...,Klinefelter Syndrome,N,This young male patient presents with a histor...,Klinefelter Syndrome,N,The prior probability of Klinefelter syndrome ...,Klinefelter Syndrome,N
510,A 9-year-old boy is brought to a pediatric psy...,Attention deficit hyperactivity disorder,ADHD,Y,ADHD,Y,ADHD,Y,ADHD,Y,...,Y,"The child's distractibility, poor academic per...","ADHD, predominantly inattentive type.",Y,"This patient's inattentiveness, talkativeness,...",Attention-Deficit/Hyperactivity Disorder,Y,he prior probability of Attention-Deficit/Hype...,Attention-Deficit/Hyperactivity Disorder.,Y
511,A 28-year-old woman is brought to the emergenc...,Bronchial rupture,Tension pneumothorax,N,Tension pneumothorax.,N,Tension pneumothorax,N,Tension pneumothorax,N,...,N,This patient's physical examination and the hi...,Tension pneumothorax.,N,This patient presents with evidence of severe ...,Traumatic Pneumothorax and Pneumomediastinum,N,The prior probability of a tension pneumothora...,Tension pneumothorax,N
512,NaN,NaN,NaN,248,NaN,206,NaN,216,NaN,195,...,385,NaN,NaN,379,NaN,NaN,384,NaN,NaN,357


### We can see that that answers that were marked as correct don't *exactly* match the "golden answer" (adhd = "attention deficit hyperactivity disorder")

### There are total counts of the correct answers on the last row of the dataframe - Lets see if the value counts align with what they report in the dataframe

In [8]:
med_qa_data['IR Grade'].value_counts()

IR Grade
N      269
Y      243
248      1
Name: count, dtype: int64

##### In the Results section of the paper, the authors state that they achieved an accuracy of 78% using DR (differential diagnoses) with GPT 4 on 498 questions. This corresponds with the column 'DR Grade.1'

In [9]:
med_qa_data['DR Grade.1'].value_counts()

DR Grade.1
Y      388
N      108
y        1
385      1
Name: count, dtype: int64

In [10]:
389 / (388 + 1 + 108)

0.7826961770623743

#### Yay this matches the results that they got

In [11]:
389 + 108

497

Maybe they counted the result row by accident? Seems like real total is 497

In [12]:
med_qa_data.shape

(513, 28)

# The important columns to us are "question" and "answer" column
"answer" column is the golden answer

## Examine the questions that are asked

In [13]:
print(med_qa_data['question'].iloc[0])

Two weeks after undergoing an emergency cardiac catherization with stenting for unstable angina pectoris, a 61-year-old man has decreased urinary output and malaise. He has type 2 diabetes mellitus and osteoarthritis of the hips. Prior to admission, his medications were insulin and naproxen. He was also started on aspirin, clopidogrel, and metoprolol after the coronary intervention. His temperature is 38°C (100.4°F), pulse is 93/min, and blood pressure is 125/85 mm Hg. Examination shows mottled, reticulated purplish discoloration of the feet. Laboratory studies show:
Hemoglobin count 14 g/dL
Leukocyte count 16,400/mm3
Segmented neutrophils 56%
Eosinophils 11%
Lymphocytes 31%
Monocytes 2%
Platelet count 260,000/mm3
Erythrocyte sedimentation rate 68 mm/h
Serum
Urea nitrogen 25 mg/dL
Creatinine 4.2 mg/dL
Renal biopsy shows intravascular spindle-shaped vacuoles. What is the most likely cause of this patient's symptoms?"


In [14]:
print(med_qa_data['question'].iloc[40])

A 14-year-old boy is brought to the physician because of an increasing difficulty in hearing over the past several months. His mother says they have to speak at a higher volume for him to understand them. He also complains of having difficulty reading his favorite books because he is not able to see the words clearly. His father received a renal transplant in his 20s. The vital signs are within normal limits. The physical examination shows no abnormalities. Laboratory studies show:
Serum
Urea nitrogen 15 mg/dL
Creatinine 1.0 mg/dL
Urine
Blood 1+
Protein 1+
RBC 15–17/hpf
WBC 1–2/hpf
The audiometry shows bilateral high-frequency sensorineural hearing loss. The ophthalmologic examination shows anterior lenticonus. What best explains these findings?


In [15]:
med_qa_data.columns

Index(['question', 'answer', 'new_IR', 'IR Grade', 'new_AR', 'AR Grade',
       'new_BR', 'BR Grade', 'new_DR', 'DR Grade', 'CoT', 'CoT.1', 'GPT-4->',
       'CoT.2', 'Rationale', 'CoT Grade', 'DR', 'Rationale.1', 'DR Grade.1',
       'IR', 'Rationale.2', 'IR Grade.1', 'AR', 'Rationale.3', 'AR Grade.1',
       'BR', 'Rationale.4', 'BR Grade.1'],
      dtype='object')

In [22]:
med_qa_data.to_csv('data/cleaned_MedQA_complete_graded_data_encoded.csv', index= False)

In [16]:
# separate out the gpt 3.5 responses from the gpt4 responses - these are all listed after the 'GPT-4->' column
gpt_4_responses = med_qa_data[['question','answer','CoT.2', 'Rationale', 'CoT Grade', 'DR', 'Rationale.1', 'DR Grade.1',
       'IR', 'Rationale.2', 'IR Grade.1', 'AR', 'Rationale.3', 'AR Grade.1',
       'BR', 'Rationale.4', 'BR Grade.1']]

# the last row is a total row, so leave it out
gpt_4_responses = gpt_4_responses.iloc[0:len(gpt_4_responses) - 1]

In [17]:
gpt_4_responses.head()

,question,answer,CoT.2,Rationale,CoT Grade,DR,Rationale.1,DR Grade.1,IR,Rationale.2,IR Grade.1,AR,Rationale.3,AR Grade.1,BR,Rationale.4,BR Grade.1
0,Two weeks after undergoing an emergency cardia...,Cholesterol embolization,Cholesterol embolization syndrome,This patient's presentation of decreased urina...,Y,Cholesterol embolization syndrome,The differential for this patient includes acu...,Y,Atheroembolic renal disease,"This patient has decreased urinary output, mal...",Y,Cholesterol embolism,The patient's recent history of cardiac cathet...,Y,Contrast-induced nephropathy,This patient's symptoms and clinical history s...,N
1,A 46-year-old man is brought to the emergency ...,Benzodiazepine intoxication,Alcoholic neuropathy,This patient presents with altered mental stat...,N,Cerebellar disease,The differential for this patient includes alc...,N,Wernicke's encephalopathy,This patient's symptoms of altered mental stat...,N,Wernicke's encephalopathy,"The patient's symptoms (somnolence, slurred sp...",N,Wernicke's encephalopathy,The prior probability of Wernicke's encephalop...,N
2,A 30-year-old African American woman comes to ...,Histoplasma capsulatum infection,Histoplasmosis,This patient has recently been hiking in Missi...,Y,Histoplasmosis,The differential diagnosis in this case includ...,Y,Histoplasmosis,"This patient presents with fever, cough, chest...",Y,Histoplasmosis,The patient's history of recent hiking in Miss...,Y,Histoplasma capsulatum,The prior probability of Histoplasmosis (a fun...,Y
3,A 67-year-old man who was diagnosed with arthr...,Psoriatic arthritis,Psoriatic Arthritis,This patient has a long history of arthritis a...,Y,Psoriatic Arthritis,The differential diagnosis for this patient in...,Y,Psoriatic Arthritis,The patient's clinical presentation suggests p...,Y,Psoriatic Arthritis,The patient presents with several features sug...,Y,Psoriatic Arthritis,The prior probability of psoriatic arthritis i...,Y
4,A one-day-old male is evaluated in the hospita...,Duodenal atresia,Duodenal atresia,The baby in question is displaying signs of in...,Y,Duodenal atresia,"This neonate presents with bilious vomiting, a...",Y,Meconium ileus,This neonate's clinical presentation of biliou...,N,Intestinal atresia,"The symptoms presented by this neonate, such a...",Y,Duodenal atresia in Down Syndrome,"In Bayesian reasoning, we begin with a prior p...",Y


In [18]:
print(gpt_4_responses['question'][100])

A 45-year-old African-American woman comes to the physician after an episode of hemoptysis. Over the past 8 months, she has had a nonproductive cough, fatigue, and a 9-kg (20-lb) weight loss. She drinks a glass of wine every night and does not smoke. She has a history of hypertension and osteoarthritis. She does not take any medications. She has never traveled out of the country. She works as an accountant. Her temperature is 37.0°C (98.6°F), pulse is 94/min, and blood pressure is 130/90 mm Hg. Lungs are clear to auscultation. An x-ray of the chest shows a 2.5-cm nodule with irregular borders at the apex of the left lung with no mediastinal or hilar enlargement. What is the most likely diagnosis?


In [19]:
# lets use the prompts that they used in the paper and see what happens

ir_prompt = """
Use symptom, signs, and laboratory disease associations to step by step deduce the correct response.
"""

In [20]:
import os
import json

from dotenv import load_dotenv
load_dotenv()

True

In [21]:
from lib.llms.chats import ChatOpenAI

In [22]:
llm_4o = ChatOpenAI(
    base_url='https://api.openai.com/v1',
    model = 'gpt-4o',
    temperature= 1,
    api_key = os.environ.get('OPENAI_API_KEY'),
    max_retries = 4
)

NameError: name 'ChatOpenAI' is not defined

In [85]:
llm_4o('Hello how are you?')

LLMInterface(id_='6e20cf48-6ea2-447a-9185-93fd235180a5', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n', content="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", source=None, channel=None, candidates=["Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?"], completion_tokens=29, total_tokens=41, prompt_tokens=12, total_cost=0, logits=[], messages=[AIMessage(id_='6110339a-9a45-4b81-b3fe-7c85cc967bf6', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationshi

In [52]:
qa_template = (
    "Diagnose the patients condition using the given passage with a short factoid answer.\n"
    "Question: {question}\n"
    "Rationale: a step-by-step deduction that identifies the correct response. {prompt}"
               )

In [53]:
qa_template

'Diagnose the patients condition using the given passage with a short factoid answer.\nQuestion: {question}\nRationale: a step-by-step deduction that identifies the correct response. {prompt}'

In [54]:
from lib.llms import PromptTemplate


prompt_template = PromptTemplate(qa_template)

AttributeError: Setting BASE_BACKEND not found

In [ ]:
print(prompt_template.populate(instructions = "these are the instructions", question = "this is my special question"))

these are the instructions
this is my special question


In [69]:


print(prompt_template.populate(question = "this is my special question", prompt = ir_prompt))

Diagnose the patients condition using the given passage with a short factoid answer.
Question:this is my special question
Rationale: a step-by-step deduction that identifies the correct response. 
Use symptom, signs, and laboratory disease associations to step by step deduce the correct response.



In [73]:
q = gpt_4_responses['question'][0]

In [74]:
print(prompt_template.populate(question = q, prompt = ir_prompt))

Diagnose the patients condition using the given passage with a short factoid answer.
Question:Two weeks after undergoing an emergency cardiac catherization with stenting for unstable angina pectoris, a 61-year-old man has decreased urinary output and malaise. He has type 2 diabetes mellitus and osteoarthritis of the hips. Prior to admission, his medications were insulin and naproxen. He was also started on aspirin, clopidogrel, and metoprolol after the coronary intervention. His temperature is 38Â°C (100.4Â°F), pulse is 93/min, and blood pressure is 125/85 mm Hg. Examination shows mottled, reticulated purplish discoloration of the feet. Laboratory studies show:
Hemoglobin count 14 g/dL
Leukocyte count 16,400/mm3
Segmented neutrophils 56%
Eosinophils 11%
Lymphocytes 31%
Monocytes 2%
Platelet count 260,000/mm3
Erythrocyte sedimentation rate 68 mm/h
Serum
Urea nitrogen 25 mg/dL
Creatinine 4.2 mg/dL
Renal biopsy shows intravascular spindle-shaped vacuoles. What is the most likely cause of 

In [86]:
res = llm_4o(prompt_template.populate(question = q, prompt = ir_prompt))

In [89]:
print(res.text)

The patient's condition is most likely due to cholesterol embolization syndrome (also known as atheroembolism). Here is a step-by-step explanation:

1. **Recent History and Symptoms**: The patient underwent cardiac catheterization and stenting two weeks ago. Following this, he's experiencing decreased urinary output and malaise. The reduced kidney function suggests possible renal injury.

2. **Skin Findings**: Mottled, reticulated purplish discoloration of the feet indicates livedo reticularis, which is commonly associated with embolic events.

3. **Laboratory Findings**: 
   - Elevated eosinophils (11%) suggesting an eosinophilic response, which is classic for cholesterol emboli.
   - Elevated leukocyte count and sedimentation rate indicate an inflammatory process.

4. **Renal Findings**: Elevated serum creatinine (4.2 mg/dL) and urea nitrogen (25 mg/dL) indicate acute kidney injury, a common occurrence in cholesterol embolization.

5. **Renal Biopsy**: The presence of intravascular s

In [172]:
from json import JSONDecodeError
from collections import defaultdict
from lib.reasoning.pipelines.answer_with_context import AnswerWithContextPipeline
from itertools import islice
import time
import asyncio
from typing import Union
import numpy as np


async def async_get_extraction(answer_with_context_pipeline: AnswerWithContextPipeline, prompt: str, evidence: str, history = []):
    try:
        extracted_answer = await answer_with_context_pipeline.ainvoke(question = prompt, evidence = evidence, history = history)

        return extracted_answer
    except Exception as e:
        print(e)
        return {"explanation": "Failed to parse",  'reason': e}

def batch_tasks(iterable, batch_size):
    iterator = iter(iterable)
    while True:
        batch = list(islice(iterator, batch_size))
        if not batch:
            break
        yield batch


async def async_extract(answer_with_context_pipeline: AnswerWithContextPipeline, prompt, evidence_arr, batch_size = 100, history = []):
    n_data = len(evidence_arr)
    
    tasks = []
    for i, evidence in enumerate(evidence_arr):
        print(f'====== \n processing {i + 1}/{n_data} files \n')

        tasks.append(async_get_extraction(answer_with_context_pipeline, prompt, evidence, history = history))

    extracted_answers = []
    needs_reprocessing = []
    for batch_number, batch in enumerate(batch_tasks(tasks, batch_size)):
        print(f'processing batch {batch_number} of size {batch_size}')
        settled_tasks = await asyncio.gather(*batch)
        for i,initial_extraction in enumerate(settled_tasks):
            idx = batch_number*batch_size + i
            if isinstance(initial_extraction, dict):
                needs_reprocessing.append(initial_extraction)
                extracted_answers.append(initial_extraction)
            else:
                try:
                    parsed_initial_extraction = json.loads(initial_extraction.content)

                    extracted_answers.append(parsed_initial_extraction)

                except JSONDecodeError as e:
                    needs_reprocessing.append({"answer": "Failed to parse", "idx": idx, 'reason': e})
                    extracted_answers.append(initial_extraction.content)
        # avoid rate limit
        time.sleep(6)
        
    return extracted_answers, needs_reprocessing

In [97]:
print('Follow the following format.\n\nQuestion: ${the question to be answered}\nRationale: a step-by-step deduction that identifies the correct response. ${Provide a step by step deduction that identifies the correct response.}\nAnswer: ${a short factoid answer, often between 1 and 5 words}'
)

Follow the following format.

Question: ${the question to be answered}
Rationale: a step-by-step deduction that identifies the correct response. ${Provide a step by step deduction that identifies the correct response.}
Answer: ${a short factoid answer, often between 1 and 5 words}


In [176]:
qa_template = (
    "Question: {context}\n"
    "Rationale: a step-by-step deduction that identifies the correct response. {question}"
    "Answer:"
    )

llm_4o = ChatOpenAI(
    base_url='https://api.openai.com/v1',
    model = 'gpt-4o',
    temperature= 1,
    api_key = os.environ.get('OPENAI_API_KEY'),
    max_retries = 4
)


answer_pipeline = AnswerWithContextPipeline(
    llm = llm_4o,
    qa_template = qa_template
    # system_prompt="you could put a system prompt here"
)

In [100]:
# lets put together the list of questions we want to ask

questions = gpt_4_responses['question'].values

In [101]:
ir_prompt = """
Use symptom, signs, and laboratory disease associations to step by step deduce the correct response.
"""

In [ ]:
answers, need_reprocess = await async_extract(answer_pipeline, ir_prompt, questions, batch_size = 100)

 processing 1/513 files 

 processing 2/513 files 

 processing 3/513 files 

 processing 4/513 files 

 processing 5/513 files 

 processing 6/513 files 

 processing 7/513 files 

 processing 8/513 files 

 processing 9/513 files 

 processing 10/513 files 

 processing 11/513 files 

 processing 12/513 files 

 processing 13/513 files 

 processing 14/513 files 

 processing 15/513 files 

 processing 16/513 files 

 processing 17/513 files 

 processing 18/513 files 

 processing 19/513 files 

 processing 20/513 files 

 processing 21/513 files 

 processing 22/513 files 

 processing 23/513 files 

 processing 24/513 files 

 processing 25/513 files 

 processing 26/513 files 

 processing 27/513 files 

 processing 28/513 files 

 processing 29/513 files 

 processing 30/513 files 

 processing 31/513 files 

 processing 32/513 files 

 processing 33/513 files 

 processing 34/513 files 

 processing 35/513 files 

 processing 36/513 files 

 processing 37/513 files 

 processin

/Users/benjamindykstra/development/health-tech-llm-code-lab/venv/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:562: RuntimeWarning: Mean of empty slice.
/Users/benjamindykstra/development/health-tech-llm-code-lab/venv/lib/python3.10/site-packages/numpy/_core/_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide


processing batch 1 of size 100
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 i

In [167]:
for ans in answers[:5]:
    print(ans)
    print('==============\n')

To determine the cause of this patient's symptoms, let's approach the problem step by step by analyzing the key symptoms, signs, laboratory findings, and the patient's history.

1. **Patient History and Procedures:**
   - The patient, a 61-year-old man with a history of type 2 diabetes mellitus and osteoarthritis, underwent an emergency cardiac catheterization with stenting for unstable angina pectoris two weeks ago.
   - Post-procedure, he was started on aspirin, clopidogrel, and metoprolol.

2. **Current Symptoms and Physical Examination:**
   - The patient presents with decreased urinary output and malaise.
   - Examination reveals mottled, reticulated purplish discoloration of the feet, which is indicative of livedo reticularis, often associated with embolic phenomena.

3. **Laboratory Findings:**
   - Leukocyte count is elevated at 16,400/mm³ with a notable eosinophilia (11%), which can suggest an allergic reaction or systemic inflammation.
   - Erythrocyte sedimentation rate (ESR

# Lets create an eval harness

In [170]:
def eval_study_responses(response_df, true_col, pred_col):
    # super simple, is answer in prediction
    no_nulls = response_df[~response_df[pred_col].isna()].reset_index()
    true_answers = no_nulls[true_col]
    pred_answers = no_nulls[pred_col]

    correct = pd.Series([a.lower() in b.lower() for a, b in zip(true_answers, pred_answers)])

    print(f'{pred_col} # correct = {correct.sum()} / {len(correct)}')

    return no_nulls[correct][[true_col, pred_col]], no_nulls[~correct][[true_col, pred_col]]


def eval_responses(response_df, true_col, llm_res):
    no_nulls = response_df[~response_df[true_col].isna()].reset_index()
    true_answers = no_nulls[true_col]
    

    correct = pd.Series([a.lower() in b.lower() for a, b in zip(true_answers, llm_res)])

    print(f'# correct = {correct.sum()} / {len(correct)}')

    # return no_nulls[correct][[true_col, pred_col]], no_nulls[~correct][[true_col, pred_col]]

In [168]:
gpt_4_responses.columns

Index(['question', 'answer', 'CoT.2', 'Rationale', 'CoT Grade', 'DR ',
       'Rationale.1', 'DR Grade.1', 'IR', 'Rationale.2', 'IR Grade.1', 'AR',
       'Rationale.3', 'AR Grade.1', 'BR', 'Rationale.4', 'BR Grade.1'],
      dtype='object')

In [171]:
study_answer_cols = ['CoT.2', 'DR ',
       'IR', 'AR','BR']

for col in study_answer_cols:

    correct, incorrect = eval_study_responses(gpt_4_responses, 'answer', col)

CoT.2 # correct = 186 / 498
DR  # correct = 209 / 497
IR # correct = 197 / 497
AR # correct = 197 / 497
BR # correct = 171 / 496


In [166]:
eval_responses(gpt_4_responses, 'answer', answers)

# correct = 256 / 512


In [173]:
dr_prompt = """
Use step by step deduction to create a differential diagnosis and then use step by step deduction to determine the correct response.
"""
dr_answers, need_reprocess = await async_extract(answer_pipeline, dr_prompt, questions, batch_size = 100)

 processing 1/513 files 

 processing 2/513 files 

 processing 3/513 files 

 processing 4/513 files 

 processing 5/513 files 

 processing 6/513 files 

 processing 7/513 files 

 processing 8/513 files 

 processing 9/513 files 

 processing 10/513 files 

 processing 11/513 files 

 processing 12/513 files 

 processing 13/513 files 

 processing 14/513 files 

 processing 15/513 files 

 processing 16/513 files 

 processing 17/513 files 

 processing 18/513 files 

 processing 19/513 files 

 processing 20/513 files 

 processing 21/513 files 

 processing 22/513 files 

 processing 23/513 files 

 processing 24/513 files 

 processing 25/513 files 

 processing 26/513 files 

 processing 27/513 files 

 processing 28/513 files 

 processing 29/513 files 

 processing 30/513 files 

 processing 31/513 files 

 processing 32/513 files 

 processing 33/513 files 

 processing 34/513 files 

 processing 35/513 files 

 processing 36/513 files 

 processing 37/513 files 

 processin

/Users/benjamindykstra/development/health-tech-llm-code-lab/venv/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:562: RuntimeWarning: Mean of empty slice.
/Users/benjamindykstra/development/health-tech-llm-code-lab/venv/lib/python3.10/site-packages/numpy/_core/_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide


processing batch 1 of size 100
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 images
Got 0 i

In [174]:
eval_responses(gpt_4_responses, 'answer', dr_answers)

# correct = 259 / 512


In [209]:
nejm_df = pd.read_csv('data/NEJM_complete_graded.csv', encoding = "utf-8")

In [210]:
nejm_df.head()

,DOI,Title,Lab Data,Answer,Initial Answer,CoT_response,CoT answer,CoT Grade,DR_response,DR answer,DR grade
0,10.1056/NEJMcpc2300894,Case 18-2023: A 19-Year-Old Woman with Dyspnea...,Hemoglobin (g/dl)\t \t9.1\t \t8.4\nHematocrit ...,Takayasu‚Äö√Ñ√¥s arteritis.,Takayasu‚Äö√Ñ√¥s arteritis.,"{\n ""id"": ""chatcmpl-7hsJNxStzqjtgaemj7C5LGihk...",Takayasu arteritis,Y,"{\n ""id"": ""chatcmpl-7hsJwbroju1Ujb4YU8uuvr29v...",Takayasu arteritis,Y
1,10.1056/NEJMcpc2201248,Case 17-2023: A 58-Year-Old Woman with Fatigue...,Hemoglobin (g/dl)\t11.1‚Äö√Ñ√¨15.9\t11.7\t14.8...,Mansonella perstans infection.,Mansonella perstans infection,"{\n ""id"": ""chatcmpl-7hsKiZNYfdJrNnJMIFpi4PG4z...",Schistosomiasis,N,"{\n ""id"": ""chatcmpl-7hsLLyDbQxIvCSa7hJDd6kvB1...",Schistosomiasis,N
2,10.1056/NEJMcpc2201237,Case 16-2023: A 13-Year-Old Boy with Depressio...,Hemoglobin (g/dl)\t \t12.8\nHematocrit (%)\t \...,Intentional guanfacine overdose.,Toxic effects of a centrally acting _2-adrener...,"{\n ""id"": ""chatcmpl-7hsLzKL2SEHds78xQje5Uzl5x...",Adrenal insufficiency,N,"{\n ""id"": ""chatcmpl-7hsMVRwBMo1fCXiPLIyXnHff2...",Dehydration,N
3,10.1056/NEJMcpc2300895,Case 15-2023: A 33-Year-Old Man with Paresthes...,NaN,Functional vitamin B12 deficiency from use of ...,Functional vitamin B12 deficiency from use of ...,"{\n ""id"": ""chatcmpl-7hsN9myJ778yIgu6rUPNYXMU3...",Nitrous oxide-induced neuropathy,Y,"{\n ""id"": ""chatcmpl-7hsNn1p0nkQzA0MynPfTtoBpU...",Nitrous oxide-induced subacute combined degene...,Y
4,10.1056/NEJMcpc2211511,Case 14-2023: A 31-Year-Old Man with Redness o...,NaN,Mpox conjunctivitis.,Mpox conjunctivitis.,"{\n ""id"": ""chatcmpl-7hsOUUAjRLFZ2nBSnTs5Hq2l9...",Monkeypox with conjunctivitis,Y,"{\n ""id"": ""chatcmpl-7hsP8QXEvkq3XgBGSBByU2nvX...",Monkeypox,Y


In [211]:
res_row = nejm_df.iloc[-1]

In [212]:
nejm_df = nejm_df[0:-1]

In [213]:
nejm_df.iloc[-1]

DOI                                          10.1056/NEJMcpc1415169
Title             Case 30-2015 — A 50-Year-Old Man with Cardioge...
Lab Data                                                        NaN
Answer            Myotonic dystrophy type 1, with associated car...
Initial Answer               Adult-onset myotonic dystrophy type 1.
CoT_response      {\n  "id": "chatcmpl-7i5HzXcISRH6UDVMYlGNYg4VL...
CoT answer                               Myotonic dystrophy type 1.
CoT Grade                                                         Y
DR_response       {\n  "id": "chatcmpl-7i5IadNWDQlPa12yVaVfmMls1...
DR answer                                Myotonic dystrophy type 1.
DR grade                                                          Y
Name: 299, dtype: object

In [214]:
nejm_df.iloc[4]['CoT_response']

'{\n  "id": "chatcmpl-7hsOUUAjRLFZ2nBSnTs5Hq2l9FykZ",\n  "object": "chat.completion",\n  "created": 1690691550,\n  "model": "gpt-4-0613",\n  "choices": [\n    {\n      "index": 0,\n      "message": {\n        "role": "assistant",\n        "content": "The patient initially presented with an itchy perianal rash and rectal pain. He was sexually active with multiple partners, which suggests a sexually transmitted infection. However, tests for common sexually transmitted infections such as HIV, gonorrhea, chlamydia, and syphilis were negative. The patient then developed a similar rash on his chest, abdomen, and back, along with fever, chills, fatigue, malaise, and headache. PCR testing of a swab specimen from the perianal vesicle detected the presence of nonvariola orthopoxvirus DNA, specifically mpox virus (MPXV; formerly known as monkeypox virus), clade II. The patient then developed redness and discomfort in his right eye, which became painful. Examination of the eye revealed conjunctiva

In [215]:
nejm_df[nejm_df['CoT_response'].isna()]

,DOI,Title,Lab Data,Answer,Initial Answer,CoT_response,CoT answer,CoT Grade,DR_response,DR answer,DR grade


In [216]:
nejm_df['CoT_response'] = nejm_df['CoT_response'].apply(json.loads)
nejm_df['DR_response'] = nejm_df['DR_response'].apply(json.loads)

In [218]:
nejm_df['CoT_response'].iloc[4]

{'id': 'chatcmpl-7hsOUUAjRLFZ2nBSnTs5Hq2l9FykZ',
 'object': 'chat.completion',
 'created': 1690691550,
 'model': 'gpt-4-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'The patient initially presented with an itchy perianal rash and rectal pain. He was sexually active with multiple partners, which suggests a sexually transmitted infection. However, tests for common sexually transmitted infections such as HIV, gonorrhea, chlamydia, and syphilis were negative. The patient then developed a similar rash on his chest, abdomen, and back, along with fever, chills, fatigue, malaise, and headache. PCR testing of a swab specimen from the perianal vesicle detected the presence of nonvariola orthopoxvirus DNA, specifically mpox virus (MPXV; formerly known as monkeypox virus), clade II. The patient then developed redness and discomfort in his right eye, which became painful. Examination of the eye revealed conjunctival injection and petechiae, but no abnormalitie

In [221]:
nejm_df.iloc[4]

DOI                                          10.1056/NEJMcpc2211511
Title             Case 14-2023: A 31-Year-Old Man with Redness o...
Lab Data                                                        NaN
Answer                                         Mpox conjunctivitis.
Initial Answer                                 Mpox conjunctivitis.
CoT_response      {'id': 'chatcmpl-7hsOUUAjRLFZ2nBSnTs5Hq2l9FykZ...
CoT answer                            Monkeypox with conjunctivitis
CoT Grade                                                         Y
DR_response       {'id': 'chatcmpl-7hsP8QXEvkq3XgBGSBByU2nvXJYAA...
DR answer                                                 Monkeypox
DR grade                                                          Y
Name: 4, dtype: object